In [21]:
pip install -e gym-TicTacToe/.

Obtaining file:///C:/Users/User/Documents/repositories/PoliTO/computational-intelligence/Laboratories/Lab10/gym-TicTacToe
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Attempting uninstall: gym-TicTacToe
    Found existing installation: gym-TicTacToe 0.0.1
    Uninstalling gym-TicTacToe-0.0.1:
      Successfully uninstalled gym-TicTacToe-0.0.1
  Running setup.py develop for gym-TicTacToe
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\User\Documents\repositories\PoliTO\computational-intelligence\Laboratories\Lab10\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [22]:
import gym_TicTacToe
import gym

# initialize the tictactoe environment
env = gym.envs.make("TTT-v0", small=-1, large=20)

In [23]:
# get 10 randomly sampled actions (between 0 and 8)
[env.action_space.sample() for _ in range(10)]

[3, 8, 6, 6, 1, 8, 5, 6, 7, 5]

In [24]:
# get 10 randomly sampled states
[env.observation_space.sample() for _ in range(10)]

[4935, 5284, 3229, 8123, 5779, 1171, 6520, 4456, 7517, 1929]

In [25]:
env.reset()
print(env.render())

╒═══╤═══╤═══╕
│ - │ - │ - │
├───┼───┼───┤
│ - │ - │ - │
├───┼───┼───┤
│ - │ - │ - │
╘═══╧═══╧═══╛


In [26]:
color = 1 # player 1
action = 4

new_state, reward, done, _ = env.step((action, color))
print(new_state, reward, done)
print(env.render())

[[0 2 0]
 [0 1 0]
 [0 0 0]] -1 False
╒═══╤═══╤═══╕
│ - │ O │ - │
├───┼───┼───┤
│ - │ X │ - │
├───┼───┼───┤
│ - │ - │ - │
╘═══╧═══╧═══╛


In [27]:
color = 2 # player 2
action = 0
new_state, reward, done, _ = env.step((action, color))
print(new_state, reward, done)
print(env.render())


[[2 2 0]
 [0 1 0]
 [0 2 0]] -1 False
╒═══╤═══╤═══╕
│ O │ O │ - │
├───┼───┼───┤
│ - │ X │ - │
├───┼───┼───┤
│ - │ O │ - │
╘═══╧═══╧═══╛


In [28]:
from collections import defaultdict 
import numpy as np
 
class DefaultDictOnKey(defaultdict):
    def __missing__(self, key):
        self[key] = self.default_factory(key) 
        return self[key] 

def init_q_table():
    d = DefaultDictOnKey( lambda key: np.array([0 for _ in key], dtype=np.float16))
    return d

def state_to_str(state):
    flatten_state = state.flatten().tolist()
    key = str(flatten_state).strip("[]").replace(',', '').replace(' ', '')

    return key
    
state_to_str(new_state)

# a = init_q_table()
# a["1001"]
# print(a["1001"])
# old = a["1001"][2] 
# a["1001"][2] = old + 0.1
# print(a["1001"]) 


'220010020'

In [29]:
state_size = env.observation_space.n
action_size = env.action_space.n
print(state_size, action_size)

8953 9


In [30]:
import numpy as np

def glie_schedule(k):
    return max(b / (b + k), eps_min)

def get_action(state, q_table, epsilon, greedy=False):
    state_q = q_table[ state_to_str(state) ]

    if greedy: # TEST -> greedy policy
        best_action_estimated = np.argmax(state_q)  # TODO: greedy w.r.t. q_grid

        return best_action_estimated

    else: # TRAINING -> epsilon-greedy policy
        if np.random.rand() < epsilon:
            # Random action
            action_chosen = np.random.choice(list(range(len(state_q))))  # TODO: choose random action with equal probability among all actions
            return action_chosen
        else:
            # Greedy action
            best_action_estimated = np.argmax(state_q)  # TODO: greedy w.r.t. q_grid
            
            return best_action_estimated

def update_q_value(old_state, action, new_state, reward, done, q_table, talk=False):
    old_state_q = q_table[ state_to_str(old_state) ]
    new_state_q = q_table[ state_to_str(new_state)]

    # Target value used for updating our current Q-function estimate at Q(old_state, action)
    if done is True:
        target_value = reward  # HINT: if the episode is finished, there is not next_state. Hence, the target value is simply the current reward.
    else:
        target_value = reward + gamma*np.max(new_state_q)

    # Update Q value
    old_q_value = old_state_q[ action ]
    q_table[ state_to_str(old_state) ][ action ] = old_q_value + alpha*(target_value - old_q_value)
    if talk:
        print("OLD STATE: ", state_to_str(old_state))
        print("ACTION: ", action)
        print("NEW STATE: ", state_to_str(new_state))
        print("OLD Q: ", old_state_q)
        print("TARGET: ", target_value)
        print("UPDATE:",  alpha*(target_value - old_q_value))
        print("NEW Q: ", q_table[ state_to_str(old_state) ])

    # TODO: IT DOESNT UPDATE QTABLE FOR SOME REASON

In [31]:
test = False

q_table = init_q_table()
print(len(q_table))


episodes = 1_000_000
test_episodes = 10
num_of_actions = env.action_space.n  

# Parameters
gamma = 0.98
alpha = 0.01
eps_min = 0.1
# eps_min = (b) / (b + ep_fin_exp)
# (1-epsmin) b = (eps_min) ep_fin_exp
# b = (eps_min / 1-eps_min) ep fin exp
b = int((eps_min/(1-eps_min)) * (episodes))


print(b)
# Training loop
ep_rewards, epl_avg = [], []
for ep in range(episodes):
    state, info, done, steps = *env.reset(), False, 0
    epsilon = glie_schedule(ep)
    turn = 0
    sum_reward = 0
    # print("="*30)
    while not done:
        action = get_action(state, q_table, epsilon, greedy=test)
        new_state, reward, done, info = env.step((action, 1))

        if not test:
            update_q_value(state, action, new_state, reward, done, q_table)#, talk=(done and reward>0))
        else:
            env.render()

            
        state = new_state
        sum_reward += reward
        steps += 1

    ep_rewards.append(sum_reward)
    epl_avg.append(np.mean(ep_rewards[ max(0, ep-10000) : ]))
    if (ep+1) % 10000 == 0:
        print("Episode {}, average reward: {:.2f}".format(ep+1, np.mean(ep_rewards[max(0, ep-10000):])))
        print("Num wins:", np.sum(np.array(ep_rewards[max(0, ep-10000):]) > 0 ))
        print('Epsilon:', epsilon)
        print("Len QTable:", len(q_table))
        print("="*50)



0
111111
Episode 10000, average reward: -18.89
Num wins: 853
Epsilon: 0.9174386920980926
Len QTable: 2982
Episode 20000, average reward: -17.51
Num wins: 1210
Epsilon: 0.8474639615589963
Len QTable: 3245
Episode 30000, average reward: -15.32
Num wins: 1771
Epsilon: 0.7874069874565941
Len QTable: 3321
Episode 40000, average reward: -13.65
Num wins: 2205
Epsilon: 0.7352987889616835
Len QTable: 3339
Episode 50000, average reward: -11.60
Num wins: 2722
Epsilon: 0.6896592390292346
Len QTable: 3352
Episode 60000, average reward: -10.12
Num wins: 3088
Epsilon: 0.6493542165858219
Len QTable: 3358
Episode 70000, average reward: -8.73
Num wins: 3420
Epsilon: 0.613500082822594
Len QTable: 3358
Episode 80000, average reward: -7.61
Num wins: 3702
Epsilon: 0.5813981476636492
Len QTable: 3359
Episode 90000, average reward: -6.01
Num wins: 4086
Epsilon: 0.5524886877828055
Len QTable: 3360
Episode 100000, average reward: -5.17
Num wins: 4294
Epsilon: 0.5263180332528066
Len QTable: 3360
Episode 110000, 

In [32]:
s = np.array([[1,1,0],[2,2,0],[0,0,0]])
print(q_table[state_to_str(s)])
get_action(s, q_table, 0, greedy=True)
print(len(q_table.items()))

for i,(k,v) in enumerate(q_table.items()):
    # print(type(v))
    if (np.array(v) > 0).any() and '0' in k:
        print(k,v)
    

[-5.207 -3.31  18.22  -3.807 -5.93   0.604 -0.892 -1.07  -1.516]
3362
000000000 [13.06  14.37  14.125 13.55  13.92  13.195 13.64  13.62  13.44 ]
000000210 [ -0.731   15.87    -0.369   -0.7285   2.719   -0.6816 -15.57   -16.03
  -0.1349]
200100210 [-1.545    1.253   -0.0673  -1.358   -0.04855 -0.0219  -1.912   -0.9795
 -0.06305]
000210000 [ -0.601    6.957    2.64   -16.38   -15.1     -0.6724   5.477   14.87
  -0.7   ]
000211020 [-0.11487 -0.09784 -0.10596 -1.545   -1.7295  -1.17    -0.06665 -0.7876
  0.4805 ]
100002000 [-15.79    14.35     3.389    4.195   -0.6084 -14.96     3.332   -0.7837
  -0.675 ]
120002010 [-1.358   -1.358   -0.08765 -0.06433 -0.0868  -1.912    0.9556  -1.7295
 -0.06354]
010000212 [  8.914 -17.73   14.31   10.72   18.22    9.37  -17.69  -17.73  -17.97 ]
000212100 [ 0.1581  1.407  18.22   -4.6    -5.79   -5.06   -5.06    0.193   0.215 ]
200000100 [-15.62     3.127    3.389   -0.5083   2.896   -0.7427 -15.43     3.479
  15.016 ]
202010100 [-1.7295  1.782  -1.358  -0

In [33]:
state, info, done, steps = *env.reset(), False, 0
state, reward, done, info = env.step((1,1))
print(state_to_str(state))
print(q_table[state_to_str(state)])
print(done)

state, reward, done, info = env.step((4,1))
print(state_to_str(state))
print(q_table[state_to_str(state)])
print(done)

state, reward, done, info = env.step((7,1))
# state, reward, done, info = env.step((8,2))
print(state_to_str(state))
print(q_table[state_to_str(state)])
print(done)

210000000
[-19.22 -19.22  11.12   8.57  15.47   9.93  12.96  15.7   10.4 ]
False
210010002
[-10.68  -11.31    8.81    4.16  -11.57    3.934   3.785  18.22  -10.68 ]
False
210010012
[0. 0. 0. 0. 0. 0. 0. 0. 0.]
True


In [53]:
import time

test = True

def is_winner(state, color: int) -> bool:
        done = False
        bool_matrix = state == color
        for ii in range(3):
            # check if three equal coins are aligned (horizontal, verical or diagonal)
            if (
                # check columns
                np.sum(bool_matrix[:, ii]) == 3
                # check rows
                or np.sum(bool_matrix[ii, :]) == 3
                # check diagonal
                or np.sum([bool_matrix[ii, ii]]) == 3
                or np.sum([bool_matrix[0, 2], bool_matrix[1, 1], bool_matrix[2, 0]])
                == 3
            ):
                done = True
                break
        return done


ep_rewards_test, epl_avg_test = [], []
wins, losses, draws = 0, 0, 0
test_episodes = 1000
for ep in range(test_episodes):
    state, info, done, steps = *env.reset(), False, 0
    turn = 0
    sum_reward = 0
    while not done:
        
        action = get_action(state, q_table, epsilon, greedy=test)
        new_state, reward, done, info = env.step((action, 1))
        # print(env.render())
        # time.sleep(.5)

        if done and is_winner(new_state, 1):
            wins += 1
        elif done and is_winner(new_state, 2):
            losses += 1
        elif done and not is_winner(new_state, 1) and not is_winner(new_state, 2):
            draws += 1

        state = new_state
        sum_reward += reward
        steps += 1
    

print("win rate:", wins/test_episodes)
print("draw rate:", draws/test_episodes)
print("lose rate:", losses/test_episodes)

win rate: 0.974
draw rate: 0.026
lose rate: 0.0


In [ ]:
test = True
env = 
def is_winner(state, color: int) -> bool:
        done = False
        bool_matrix = state == color
        for ii in range(3):
            # check if three equal coins are aligned (horizontal, verical or diagonal)
            if (
                # check columns
                np.sum(bool_matrix[:, ii]) == 3
                # check rows
                or np.sum(bool_matrix[ii, :]) == 3
                # check diagonal
                or np.sum([bool_matrix[ii, ii]]) == 3
                or np.sum([bool_matrix[0, 2], bool_matrix[1, 1], bool_matrix[2, 0]])
                == 3
            ):
                done = True
                break
        return done


ep_rewards_test, epl_avg_test = [], []
wins, losses, draws = 0, 0, 0
test_episodes = 100_000
for ep in range(test_episodes):
    state, info, done, steps = *env.reset(), False, 0
    turn = 0
    sum_reward = 0
    while not done:
        
        action = get_action(state, q_table, epsilon, greedy=test)
        new_state, reward, done, info = env.step((action, 1))


        if done and is_winner(new_state, 1):
            wins += 1
        elif done and is_winner(new_state, 2):
            losses += 1
        elif done and not is_winner(new_state, 1) and not is_winner(new_state, 2):
            draws += 1

        state = new_state
        sum_reward += reward
        steps += 1
    

print("win rate:", wins/test_episodes)
print("draw rate:", draws/test_episodes)
print("lose rate:", losses/test_episodes)